# Charts

In [21]:
import numpy as np
import pandas as pd
from siuba import *
from calitp import *
import intake
import data_prep
import _utils
import shared_utils
import altair as alt
import altair_saver
from shared_utils import altair_utils 
pd.options.display.max_rows = 100
pd.options.display.float_format = "{:.0f}".format
pd.options.display.float_format = '{:,}'.format

In [22]:
#Natalie's cleaned df 
agg_parquet = pd.read_parquet('5311_agg.parquet')


In [73]:
agg_parquet.gtfs_realtime_status.unique()

array(['RT OK', 'RT Incomplete', None], dtype=object)

In [74]:
agg_parquet.gtfs_static_status.unique()

array(['Static OK', 'Static Incomplete', None], dtype=object)

In [83]:
def gtfs_status_indicator(df):   
    if (df['gtfs_static_status'] == 'Static OK') and (df['gtfs_realtime_status'] == "RT OK"):
        return 'Both static & RT OK'
    elif (df['gtfs_static_status'] == 'Static Incomplete') and (df['gtfs_realtime_status'] == "RT OK"):
        return 'Only RT'
    elif (df['gtfs_static_status'] == 'Static OK') and (df['gtfs_realtime_status'] == "RT Incomplete"):
        return 'Only Static'
    else: 
        return "None"

In [96]:
agg_parquet['GTFS_Status'] = agg_parquet.apply(gtfs_status_indicator, axis = 1)
agg_parquet[['organization_name','gtfs_static_status','gtfs_realtime_status','GTFS_Status']]

,organization_name,gtfs_static_status,gtfs_realtime_status,GTFS_Status
0,Madera County,Static OK,RT OK,Both static & RT OK
1,Fresno County Rural Transit Agency,Static Incomplete,RT Incomplete,None
2,Yuba-Sutter Transit Authority,Static OK,RT Incomplete,Only Static
3,City of Arcata,Static OK,RT OK,Both static & RT OK
4,City of Ojai,Static OK,RT OK,Both static & RT OK
5,San Diego Metropolitan Transit System,Static OK,RT Incomplete,Only Static
6,Humboldt Transit Authority,Static OK,RT OK,Both static & RT OK
7,City of Arvin,Static OK,RT Incomplete,Only Static
8,City of Auburn,Static OK,RT Incomplete,Only Static
9,City of California City,Static OK,RT Incomplete,Only Static


In [23]:
#Original DF
BC_GTFS_NTD2 = data_prep.merged_dataframe()

/home/jovyan/data-analyses/5311/data_prep.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep.py:70: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/jovyan/data-analyses/5311/data_prep.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

In [24]:
#Grabbing only one row for each agency with original df
Aggregate1 = BC_GTFS_NTD2.groupby(['organization_name','reporter_type','GTFS', 'fleet_size', 'Is_Agency_In_BC_Only_1_means_Yes']).agg({'total_vehicles':'max',
       'average_age_of_fleet__in_years_':'max','doors_sum':'max','_0_9':'max', '_10_12':'max', '_13_15':'max', '_16_20':'max', '_21_25':'max',
       '_26_30':'max', '_31_60':'max', '_60plus':'max',})


In [68]:
Aggregate1 = Aggregate1.reset_index()

# GTFS by Orgs 

In [91]:
GTFS_orgs = agg_parquet.groupby(['GTFS_Status']).agg({'organization_name':'nunique'}).reset_index().rename(columns = {'organization_name': 'Count_of_Organizations'}) 
GTFS_orgs

,GTFS_Status,Count_of_Organizations
0,Both static & RT OK,13
1,None,27
2,Only RT,2
3,Only Static,45


In [92]:
_utils.basic_bar_chart(GTFS_orgs, 'GTFS_Status','Count_of_Organizations', 'GTFS_Status') 

alt.Chart(...)

# GTFS by Orgs (Super Rough Estimate)

In [69]:
GTFS_orgs_rough = Aggregate1.groupby(['GTFS']).agg({'organization_name':'nunique'})
GTFS_orgs_rough = GTFS_orgs_rough.reset_index()
GTFS_orgs_rough = GTFS_orgs_rough.rename(columns = {'organization_name': 'Count_of_Organizations'}) 
GTFS_orgs_rough

,GTFS,Count_of_Organizations
0,Long-term solution needed,3
1,Needed,15
2,None,14
3,Ok,54
4,Research,1


In [70]:
_utils.basic_bar_chart(GTFS_orgs_rough, 'GTFS','Count_of_Organizations', 'GTFS') 

alt.Chart(...)

#  Organizations within each district

In [25]:
agg_parquet.head(1)

,organization_name,reporter_type,fleet_size,ntd_id,itp_id,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,Automobiles,Bus,Other,Train,Van,automobiles_door,bus_doors,van_doors,train_doors,doors_sum,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_16plus,_60plus,caltrans_district,mpo_rtpa,planning_authority,gtfs_static_status,gtfs_realtime_status
0,Madera County,Rural Reporter,Medium,9R02-91005,188.0,"150,308.83333333334","156,798.11111111112","110,124.74055555556","40,184.09277777778",0.0,16.0,5.4642857145,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,16.0,0.0,16.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,06 - Fresno,None,None,Static OK,RT OK


In [26]:
Orgs_in_district = agg_parquet.groupby(['caltrans_district']).agg({'organization_name':'nunique'}).reset_index().rename(columns = {'organization_name':'count_of_organizations'})
Orgs_in_district = Orgs_in_district.sort_values(by='count_of_organizations', ascending=False)
Orgs_in_district

,caltrans_district,count_of_organizations
5,06 - Fresno,13
2,03 - Marysville,7
3,04 - Oakland,7
7,08 - San Bernardino,7
4,05 - San Luis Obispo,6
0,01 - Eureka,5
9,10 - Stockton,4
1,02 - Redding,3
8,09 - Bishop,3
10,11 - San Diego,3


In [27]:
_utils.basic_bar_chart(Orgs_in_district,'caltrans_district','count_of_organizations','caltrans_district') 

alt.Chart(...)

#  GTFS Status Charts
* One for vehicles over 9
* One for vehicles over 15

In [28]:
vehicles_gtfs = _utils.aggregation_one(Aggregate1, 'GTFS')
vehicles_gtfs

,GTFS,vehicles_older_than_9,vehicles_older_than_15,vehicles_0_to_9,vehicles_percent_older_than_9,vehicles_percent_older_than_15,vehicles_percent_0_to_9
0,Long-term solution needed,7.0,1.0,24.0,0.34347399411187435,0.13351134846461948,0.4840661557079467
1,Needed,34.0,7.0,122.0,1.6683022571148183,0.9345794392523363,2.4606696248487294
2,None,51.0,5.0,191.0,2.502453385672228,0.6675567423230975,3.8523598225090763
3,Ok,"1,901.0",720.0,"4,470.0",93.27772325809617,96.12817089452604,90.15732150060508
4,Research,45.0,16.0,151.0,2.2080471050049066,2.1361815754339117,3.0455828963291647


In [29]:
_utils.basic_bar_chart(vehicles_gtfs,'GTFS','vehicles_older_than_9','GTFS') 

alt.Chart(...)

In [30]:
_utils.basic_bar_chart(vehicles_gtfs,'GTFS','vehicles_percent_older_than_9','GTFS') 

alt.Chart(...)

In [31]:
_utils.basic_bar_chart(vehicles_gtfs,'GTFS','vehicles_older_than_15', 'GTFS') 

alt.Chart(...)

# Fleet size 

In [32]:
fleet = _utils.aggregation_one(agg_parquet,'fleet_size') 
fleet

,fleet_size,vehicles_older_than_9,vehicles_older_than_15,vehicles_0_to_9,vehicles_percent_older_than_9,vehicles_percent_older_than_15,vehicles_percent_0_to_9
0,Large,"1,757.0",696.0,"3,975.0",86.21197252208047,92.92389853137517,80.17345703912868
1,Medium,255.0,48.0,903.0,12.512266928361138,6.408544726301736,18.212989108511497
2,No Info,6.0,0.0,12.0,0.2944062806673209,0.0,0.24203307785397335
3,Small,20.0,5.0,68.0,0.9813542688910697,0.6675567423230975,1.371520774505849


In [33]:
# don't need no info
fleet =  fleet.loc[fleet['fleet_size'] != 'No Info'] 

In [34]:
_utils.basic_bar_chart(fleet,'fleet_size','vehicles_older_than_9', 'fleet_size') 

alt.Chart(...)

In [35]:
_utils.basic_bar_chart(fleet,'fleet_size','vehicles_older_than_15', 'fleet_size') 

alt.Chart(...)

In [36]:
Orgs = agg_parquet.groupby(['gtfs_static_status']).agg({'organization_name':'nunique'})
Orgs = Orgs.reset_index()
Orgs = Orgs.rename(columns = {'organization_name': 'Organizations'}) 
_utils.basic_bar_chart(Orgs, 'gtfs_static_status','Organizations', 'gtfs_static_status') 


alt.Chart(...)

### Fleet Size by agencies

In [37]:
Fleet = agg_parquet.groupby(['fleet_size']).agg({'organization_name':'nunique'}).reset_index()
Fleet

,fleet_size,organization_name
0,Large,20
1,Medium,38
2,No Info,11
3,Small,18


![testing](./chart_outputs/bar_reporter_type_by_Count of Agencies.png)

In [38]:
agg_parquet.head(1)

,organization_name,reporter_type,fleet_size,ntd_id,itp_id,allocationamount,encumbered_amount,expendedamount,activebalance,closedoutbalance,total_vehicles,average_age_of_fleet__in_years_,average_lifetime_miles_per_vehicle,Automobiles,Bus,Other,Train,Van,automobiles_door,bus_doors,van_doors,train_doors,doors_sum,_0_9,_10_12,_13_15,_16_20,_21_25,_26_30,_31_60,_16plus,_60plus,caltrans_district,mpo_rtpa,planning_authority,gtfs_static_status,gtfs_realtime_status,vehicles_older_than_9,vehicles_older_than_15
0,Madera County,Rural Reporter,Medium,9R02-91005,188.0,"150,308.83333333334","156,798.11111111112","110,124.74055555556","40,184.09277777778",0.0,16.0,5.4642857145,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,16.0,0.0,16.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,06 - Fresno,None,None,Static OK,RT OK,0.0,0.0


In [39]:
Reporter_type_agg = agg_parquet.groupby(['reporter_type']).agg({'organization_name':'nunique'})
Reporter_type_agg = Reporter_type_agg.reset_index()
Reporter_type_agg = Reporter_type_agg.rename(columns = {'organization_name':'Count_of_Agencies'})
_utils.basic_bar_chart(Reporter_type_agg,'reporter_type','Count_of_Agencies', 'reporter_type') 

alt.Chart(...)

In [40]:
fleet_size_avg_age = agg_parquet.groupby(['fleet_size']).agg({'average_age_of_fleet__in_years_':'mean'})
fleet_size_avg_age

,average_age_of_fleet__in_years_
fleet_size,
Large,7.971572030532262
Medium,6.5403448314342105
No Info,7.6375
Small,6.904166666569444


In [42]:
age = agg_parquet
age  =  age.loc[age['fleet_size'] != 'No Info'] 
age = age.groupby(['gtfs_realtime_status','fleet_size']).agg({'_0_9':'sum', 'vehicles_older_than_9':'sum','average_age_of_fleet__in_years_':'mean'})
age

_0_9  vehicles_older_than_9  \
gtfs_realtime_status fleet_size                                  
RT Incomplete        Large      1,643.0                  823.0   
                     Medium       587.0                  161.0   
                     Small         46.0                   12.0   
RT OK                Large      1,916.0                  793.0   
                     Medium       195.0                   62.0   
                     Small          6.0                    1.0   

                                 average_age_of_fleet__in_years_  
gtfs_realtime_status fleet_size                                   
RT Incomplete        Large                        8.029637887225  
                     Medium                    6.674995801576666  
                     Small                     6.498611110833333  
RT OK                Large                     8.746050001311223  
                     Medium                    6.527341868404762  
                     Small                                   7.5

# Looking at orgs that spent the most..

In [58]:
Most_Money = agg_parquet.groupby(['organization_name','gtfs_realtime_status','gtfs_static_status']).agg({'allocationamount':'sum'})

In [59]:
Most_Money.sort_values('allocationamount')

,,,allocationamount
organization_name,gtfs_realtime_status,gtfs_static_status,
City of Taft,RT Incomplete,Static OK,"31,098.5"
City of Needles,RT Incomplete,Static OK,"40,160.833333333336"
City of Tehachapi,RT Incomplete,Static OK,"40,686.166666666664"
City of Woodlake,RT Incomplete,Static OK,"44,573.0"
City of California City,RT Incomplete,Static OK,"46,001.0"
Sierra County Transportation Commission,RT Incomplete,Static OK,"48,000.0"
City of McFarland,RT Incomplete,Static OK,"48,250.166666666664"
City of Shafter,RT Incomplete,Static OK,"60,676.0"
City of Guadalupe,RT Incomplete,Static OK,"62,625.666666666664"


In [60]:
Top_ten = Most_Money.sort_values('allocationamount').tail(10)
Top_ten =Top_ten.rename(columns = {'allocationamount': '5311_funds_received'})
Top_ten = Top_ten.reset_index()
Top_ten

,organization_name,gtfs_realtime_status,gtfs_static_status,5311_funds_received
0,San Benito County Local Transportation Authority,RT Incomplete,Static OK,"316,169.0"
1,Tehama County,RT Incomplete,Static OK,"363,043.1666666667"
2,Mendocino Transit Authority,RT Incomplete,Static OK,"382,165.0909090909"
3,North County Transit District,RT OK,Static OK,"440,263.5"
4,Butte County Association of Governments,RT Incomplete,Static OK,"490,548.9166666667"
5,El Dorado County Transit Authority,RT Incomplete,Static OK,"508,602.6666666667"
6,Riverside Transit Agency,RT OK,Static OK,"530,467.5"
7,Transit Joint Powers Authority for Merced County,RT Incomplete,Static OK,"536,006.3333333334"
8,Victor Valley Transit Authority,RT OK,Static OK,"665,100.1666666666"
9,Fresno County Rural Transit Agency,RT Incomplete,Static Incomplete,"877,618.7"


In [61]:
Top_ten.gtfs_realtime_status.value_counts()

RT Incomplete    7
RT OK            3
Name: gtfs_realtime_status, dtype: int64

In [62]:
Top_ten.gtfs_static_status.value_counts()

Static OK            9
Static Incomplete    1
Name: gtfs_static_status, dtype: int64

In [46]:
Top_ten_agencies = Top_ten['organization_name'].tolist()
Top_ten_agencies

['San Benito County Local Transportation Authority',
 'Tehama County',
 'Mendocino Transit Authority',
 'North County Transit District',
 'Butte County Association of Governments',
 'El Dorado County Transit Authority',
 'Riverside Transit Agency',
 'Transit Joint Powers Authority for Merced County',
 'Victor Valley Transit Authority',
 'Fresno County Rural Transit Agency']

In [47]:
Year_look = BC_GTFS_NTD2.groupby(['grant_fiscal_year','organization_name']).agg({'allocationamount':'sum'})
Year_look = Year_look.reset_index()

Year_look = Year_look[Year_look.organization_name.isin(Top_ten_agencies)]
Year_look.groupby(['organization_name','grant_fiscal_year']).agg({'allocationamount':'sum'})

allocationamount
organization_name                                grant_fiscal_year                  
Butte County Association of Governments          2016                      650,743.0
                                                 2017                      957,263.0
                                                 2018                      894,274.0
                                                 2019                    1,281,072.0
                                                 2020                    1,037,498.0
                                                 2021                    1,065,737.0
El Dorado County Transit Authority               2016                      474,917.0
                                                 2017                      479,675.0
                                                 2018                      490,631.0
                                                 2019                      509,322.0
                                                 2020                      538,231.0
                                                 2021                      558,840.0
Fresno County Rural Transit Agency               2013                       78,577.0
                                                 2016                    1,473,348.0
                                                 2017                    1,488,110.0
                                                 2018                      810,576.0
                                                 2019                    1,522,098.0
                                                 2020                    1,669,771.0
                                                 2021                    1,733,707.0
Mendocino Transit Authority                      2016                      469,217.0
                                                 2017                      639,908.0
                                                 2018                      607,575.0
                                                 2019                      803,210.0
                                                 2020                      831,772.0
                                                 2021                      852,134.0
North County Transit District                    2016                      411,177.0
                                                 2017                      415,297.0
                                                 2018                      424,310.0
                                                 2019                      440,965.0
                                                 2020                      465,994.0
                                                 2021                      483,838.0
Riverside Transit Agency                         2016                      550,261.0
                                                 2017                      555,774.0
                                                 2018                      568,468.0
                                                 2019                      478,221.0
                                                 2020                      505,365.0
                                                 2021                      524,716.0
San Benito County Local Transportation Authority 2016                      295,228.0
                                                 2017                      298,186.0
                                                 2018                      304,997.0
                                                 2019                      316,616.0
                                                 2020                      334,588.0
                                                 2021                      347,399.0
Tehama County                                    2016                      338,998.0
                                                 2017                      342,394.0
                                                 2018                    

In [48]:
_utils.basic_bar_chart(Top_ten,'organization_name','5311_funds_received', 'organization_name') 

alt.Chart(...)

In [49]:
bc_funds = BC_GTFS_NTD2.groupby(['funding_program']).agg({'organization_name':'nunique','allocationamount':'sum'})
bc_funds = bc_funds.rename(columns = {'organization_name':'Count_of_Organizations','allocationamount':'total_sum'}).reset_index()
bc_funds.sort_values(by =['total_sum'])

_utils.basic_bar_chart(bc_funds,'funding_program','total_sum', 'funding_program') 

alt.Chart(...)

In [50]:
bc_funds

,funding_program,Count_of_Organizations,total_sum
0,5311(f) Cont,21,"7,627,280.0"
1,5311(f) Round 2,20,"6,203,361.0"
2,CMAQ (FTA 5311),6,"2,532,205.0"
3,Section 5311,81,"129,415,424.28"
4,Section 5311(f),22,"18,753,232.0"


In [51]:
129/164

0.7865853658536586

In [52]:
BC_GTFS_NTD2.columns

Index(['grant_fiscal_year', 'funding_program', 'grant_number', 'project_year',
       'organization_name', 'upin', 'description', 'ali', 'contract_number',
       'allocationamount', 'encumbered_amount', 'expendedamount',
       'activebalance', 'closedoutbalance', 'project_status', 'ntd_id',
       'reporter_type', '_0_9', '_10_12', '_13_15', '_16_20', '_21_25',
       '_26_30', '_31_60', '_60plus', 'total_vehicles',
       'average_age_of_fleet__in_years_', 'average_lifetime_miles_per_vehicle',
       'sum_15plus', 'Automobiles', 'Bus', 'Other', 'Service', 'Train', 'Van',
       'automobiles_door', 'bus_doors', 'train_doors', 'van_doors',
       'doors_sum', 'itp_id', '_merge', 'Is_Agency_In_BC_Only_1_means_Yes',
       'GTFS', 'fleet_size'],
      dtype='object')

In [53]:
project_money = BC_GTFS_NTD2.groupby(['description']).agg({'upin':'nunique','allocationamount':'sum'}).rename(columns = {'allocationamount': '5311_funds_received'})

In [54]:

project_money = project_money.sort_values('5311_funds_received').tail(10).reset_index()

In [55]:
_utils.basic_bar_chart(project_money,'description','5311_funds_received', 'description') 

alt.Chart(...)